In [1]:
import numpy as np
import pandas as pd
import xlsxwriter

import wmfdata as wmf
from wmfdata.utils import get_dblist

# Parameters

In [10]:
# Data will be generated for the 12 months ending with and including SNAPSHOT
# This is also the mediawiki_history snapshot necessary for the calculations.
SNAPSHOT = "2021-12"

# An ordered list of wikis for which to output each metric after it is generated. 
# This is useful for spot checking the values generated against the previous snapshot.
WIKIS_TO_CHECK = [
    "enwiki",
    "eswiki",
    "jawiki",
    "dewiki",
    "frwiki",
    "ruwiki"
]

# Date manipulation

In [3]:
snapshot = pd.Period(SNAPSHOT)
start = (snapshot - 11).start_time
end = (snapshot + 1).start_time
# Used to name the output files
file_stem = snapshot.strftime("%b %Y")

# Start included, end excluded
query_vars = dict(
    snapshot=snapshot.strftime("%Y-%m"),
    start=start.strftime('%Y-%m-%d'), 
    end=end.strftime('%Y-%m-%d'),
    ym_start=start.strftime("%Y-%m"),
    ym_end=end.strftime("%Y-%m"),
    pv_start=start.strftime("%Y%m"),
    pv_end=end.strftime("%Y%m"),
    # New editor retention needs different time boundaries, since we define retention
    # in a given month as the status of the new users who registered two months prior
    ner_cohort_start=(snapshot - 14).strftime("%Y-%m"),
    ner_cohort_end=(snapshot - 1).strftime("%Y-%m")
)

# List of wikis

In [4]:
# Gather all content wikis
wikis = wmf.spark.run("""
SELECT
  database_code,
  database_group AS project_code,
  language_code,
  domain_name,
  language_name,
  english_name as wiki_name
FROM canonical_data.wikis
WHERE
  database_group IN (
    "commons", "incubator", "foundation", "mediawiki", "meta", "sources",
    "species","wikibooks", "wikidata", "wikinews", "wikipedia", "wikiquote",
    "wikisource", "wikiversity", "wikivoyage", "wiktionary"
  )
  AND status = "open"
  AND visibility = "public"
  AND editability = "public"
""", session_type="yarn-large")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


PYSPARK_PYTHON=/usr/lib/anaconda-wmf/bin/python3


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark2/jars/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/21 18:55:50 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/01/21 18:55:50 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
22/01/21 18:55:50 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
22/01/21 18:55:50 

# Check for missing wikis in mediawiki_history

Sometimes, wikis are not added to mediawiki_history after they're created (e.g. [T299548](https://phabricator.wikimedia.org/T299548), [T220456](https://phabricator.wikimedia.org/T220456)). Let's check for that.

In [5]:
mwh_wikis = wmf.spark.run("""
SELECT DISTINCT wiki_db AS database_code
FROM wmf.mediawiki_history
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
22/01/21 18:56:20 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
22/01/21 18:57:58 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


In [6]:
mwh_missing_wikis = set(wikis["database_code"]) - set(mwh_wikis["database_code"])
mwh_missing_wikis

{'altwiki',
 'amiwiki',
 'banwikisource',
 'bclwiktionary',
 'dagwiki',
 'diqwiktionary',
 'jvwikisource',
 'lmowiktionary',
 'madwiki',
 'mniwiki',
 'mniwiktionary',
 'mnwwiktionary',
 'niawiki',
 'niawiktionary',
 'pwnwiki',
 'shiwiki',
 'skrwiki',
 'skrwiktionary',
 'smnwiki',
 'taywiki',
 'trvwiki',
 'wawikisource'}

These missing wikis should be reported to Data Engineering. In the meantime, let's continue generating the data for the remaining wikis.

In [7]:
wikis = wikis.drop(
    index=wikis.query("database_code in @mwh_missing_wikis").index
)

# Data

In [12]:
def merge_in(df, on="database_code"):
    global wikis
    wikis = pd.merge(wikis, df, how="left", on=on).fillna(0)

wikis_to_check = (
    pd.DataFrame({"database_code": WIKIS_TO_CHECK})
    .merge(wikis, how="left", on="database_code")
)

def check(df, index_col="database_code"):
    indexes = wikis_to_check[index_col].to_frame()
    return indexes.merge(df, how="left", on=index_col)

## Monthly active editors

In [9]:
mae = wmf.spark.run("""
SELECT
    wiki AS database_code,
    COUNT(*) / 12 AS monthly_active_editors,
    SUM(
        CAST(TRUNC(user_registration, 'MM') = TRUNC(month, 'MM') AS INT)
    )/ 12 AS monthly_new_active_editors
FROM neilpquinn.editor_month
WHERE
    content_edits >= 5 
    AND month >= "{start}" 
    AND month < "{end}" 
    AND user_id != 0 
    AND bot_by_group = FALSE 
    AND (
        user_name NOT REGEXP "bot\\b"
        OR user_name IN ("Paucabot", "Niabot", "Marbot")    
    )    
GROUP BY wiki
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
22/01/21 19:03:15 WARN TransportChannelHandler: Exception in connection from /10.64.5.14:32942
java.io.IOException: Connection reset by peer
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:379)
	at io.netty.buffer.PooledUnsafeDirectByteBuf.setBytes(PooledUnsafeDirectByteBuf.java:288)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1106)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:343)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:123)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:645)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOptimized(NioEventL

In [13]:
check(mae)

,database_code,monthly_active_editors,monthly_new_active_editors
0,enwiki,32804.583333,4946.416667
1,eswiki,4644.333333,1036.583333
2,jawiki,5389.333333,876.500000
3,dewiki,5523.833333,434.000000
4,frwiki,5199.333333,782.000000
5,ruwiki,3630.833333,543.916667


In [14]:
merge_in(mae)

## Unique devices

In [15]:
ud = wmf.spark.run("""
SELECT
    -- Strip mobile subdomains so mobile and desktop sites are combined. 
    REGEXP_REPLACE(
        REGEXP_REPLACE(
            -- The canonical domains for Wikidata and MediaWiki.org start with `www`, which 
            -- gets _replaced_ by the mobile subdomain. Combine the two possibilites for each site.
            REGEXP_REPLACE(
                REGEXP_REPLACE(domain, "^m\\\\.wikidata", "www.wikidata"),
            "^m\\\\.mediawiki", "www.mediawiki"),
        "^m\\\\.", ""),
    "\\\\.m\\\\.", ".") AS domain_name,
    SUM(uniques_estimate) / 12 AS monthly_unique_devices,
    SUM(IF(
        (domain REGEXP "^m\\\\." OR  domain REGEXP "\\\\.m\\\\."),
        uniques_estimate,
        0
    )) / SUM(uniques_estimate) AS mobile_unique_devices
FROM wmf.unique_devices_per_domain_monthly
WHERE 
    CONCAT(year, LPAD(month, 2, "0")) >= "{pv_start}" 
    AND CONCAT(year, LPAD(month, 2, "0")) < "{pv_end}"  
GROUP BY    
    REGEXP_REPLACE(
        REGEXP_REPLACE(
            -- The canonical domains for Wikidata and MediaWiki.org start with `www`, which 
            -- gets _replaced_ by the mobile subdomain. Combine the two possibilites for each site.
            REGEXP_REPLACE(
                REGEXP_REPLACE(domain, "^m\\\\.wikidata", "www.wikidata"),
            "^m\\\\.mediawiki", "www.mediawiki"),
        "^m\\\\.", ""),
    "\\\\.m\\\\.", ".")
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [16]:
check(ud, index_col="domain_name")

,domain_name,monthly_unique_devices,mobile_unique_devices
0,en.wikipedia.org,8.093526e+08,0.696283
1,es.wikipedia.org,1.573251e+08,0.708869
2,ja.wikipedia.org,1.075370e+08,0.734168
3,de.wikipedia.org,1.047216e+08,0.605649
4,fr.wikipedia.org,9.424073e+07,0.653896
5,ru.wikipedia.org,9.900818e+07,0.654718


In [17]:
merge_in(ud, on="domain_name")

## Overall size rank

In [18]:
size = np.sqrt(wikis["monthly_unique_devices"] * wikis["monthly_active_editors"])
rank = size.rank(method="min", na_option="bottom", ascending=False)
wikis["overall_size_rank"] = rank

## New editor retention

In [19]:
ner = wmf.spark.run(
"""
SELECT 
    wiki AS database_code,
    SUM(CAST(2nd_month_edits >= 1 AS INT))
        / SUM(CAST(1st_month_edits >= 1 AS INT)) AS second_month_new_editor_retention
FROM cchen.new_editors
WHERE 
    cohort >= "{ner_cohort_start}" and
    cohort < "{ner_cohort_end}"
GROUP BY wiki
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [20]:
check(ner, "database_code")

,database_code,second_month_new_editor_retention
0,enwiki,0.076538
1,eswiki,0.050753
2,jawiki,0.106032
3,dewiki,0.074379
4,frwiki,0.063499
5,ruwiki,0.062890


In [21]:
merge_in(ner)

## Mobile editing proportion

In [22]:
mep = wmf.spark.run(
"""
SELECT
    CONCAT(project, ".org") AS domain_name,
    SUM(CASE WHEN ARRAY_CONTAINS(revision_tags, "mobile edit") THEN edit_count END)
        / SUM(edit_count) AS mobile_edits
FROM wmf.edit_hourly
WHERE
    ts >= "{start}" 
    AND ts < "{end}"
    AND snapshot = "{snapshot}"
    AND NOT user_is_bot
GROUP BY CONCAT(project, ".org")
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [23]:
check(mep, index_col="domain_name")

,domain_name,mobile_edits
0,en.wikipedia.org,0.144092
1,es.wikipedia.org,0.239887
2,ja.wikipedia.org,0.232572
3,de.wikipedia.org,0.050405
4,fr.wikipedia.org,0.098942
5,ru.wikipedia.org,0.106020


In [24]:
merge_in(mep, on="domain_name")

## Bot editing proportion

In [25]:
bep = wmf.spark.run("""
SELECT
   CONCAT(project, ".org") AS domain_name,
   SUM(CASE WHEN user_is_bot THEN edit_count END)
        / SUM(edit_count) AS bot_edits
FROM wmf.edit_hourly
WHERE
    ts  >= "{start}" 
    AND ts  < "{end}"
    AND snapshot = "{snapshot}"
GROUP BY CONCAT(project, ".org")
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [26]:
check(bep, index_col="domain_name")

,domain_name,bot_edits
0,en.wikipedia.org,0.175525
1,es.wikipedia.org,0.103631
2,ja.wikipedia.org,0.115351
3,de.wikipedia.org,0.121770
4,fr.wikipedia.org,0.206058
5,ru.wikipedia.org,0.199857


In [27]:
merge_in(bep, on="domain_name")

## Anonymous editing proportion

In [28]:
aep = wmf.spark.run("""
SELECT
   CONCAT(project, ".org") AS domain_name,
   SUM(CASE WHEN user_is_anonymous THEN edit_count END) / SUM(edit_count) AS anonymous_edits
FROM wmf.edit_hourly
WHERE
    ts >= "{start}" 
    AND ts < "{end}"
    AND snapshot = "{snapshot}"
    AND NOT user_is_bot
GROUP BY CONCAT(project, ".org")
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [29]:
check(aep, index_col="domain_name")

,domain_name,anonymous_edits
0,en.wikipedia.org,0.170487
1,es.wikipedia.org,0.270625
2,ja.wikipedia.org,0.241580
3,de.wikipedia.org,0.089402
4,fr.wikipedia.org,0.125333
5,ru.wikipedia.org,0.186179


In [30]:
merge_in(aep, on="domain_name")

## Majority-mobile editors proportion

In [31]:
mmep = wmf.spark.run("""
WITH user AS (
    SELECT 
        wiki_db AS database_code,
        SUM(CAST(ARRAY_CONTAINS(revision_tags, "mobile edit") AS INT))
            / COUNT(*) AS mobile_editing_proportion
    FROM wmf.mediawiki_history
    WHERE
        event_entity = "revision" 
        AND event_type = "create" 
        AND snapshot = "{snapshot}" 
        AND event_timestamp >= "{start}" 
        AND event_timestamp < "{end}"
        AND NOT event_user_is_anonymous
        AND SIZE(event_user_is_bot_by_historical) = 0
    GROUP BY
        wiki_db,
        event_user_text
)
SELECT
   database_code,
   SUM(CAST(mobile_editing_proportion > 0.5 AS INT))
       / COUNT(*) AS majority_mobile_editors
FROM user
GROUP BY database_code
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [32]:
check(mmep)

,database_code,majority_mobile_editors
0,enwiki,0.261189
1,eswiki,0.300681
2,jawiki,0.296639
3,dewiki,0.147814
4,frwiki,0.179282
5,ruwiki,0.296091


In [33]:
merge_in(mmep)

## Revert rate

In [34]:
rr = wmf.spark.run("""
    SELECT
        wiki_db AS database_code,
        SUM(CAST(revision_is_identity_reverted AS INT)) / COUNT(*) AS revert_rate
    FROM
        wmf.mediawiki_history
    WHERE
        event_entity = "revision" 
        AND event_type = "create" 
        AND snapshot = "{snapshot}" 
        AND event_timestamp >= "{start}" 
        AND event_timestamp < "{end}" 
        AND SIZE(event_user_is_bot_by_historical) = 0 
    GROUP BY wiki_db
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [36]:
check(rr)

,database_code,revert_rate
0,enwiki,0.110091
1,eswiki,0.186721
2,jawiki,0.068036
3,dewiki,0.075174
4,frwiki,0.070056
5,ruwiki,0.104664


In [37]:
merge_in(rr)

## Pageviews

In [38]:
pv = wmf.spark.run("""
SELECT
    IF(
        project IN ("mediawiki", "wikidata"),
        CONCAT("www.", project, ".org"),
        CONCAT(project, ".org")
    ) AS domain_name, 
    SUM(view_count) / 12 AS monthly_pageviews,
    SUM(CASE WHEN access_method = "mobile web" THEN view_count END)
        / SUM(view_count) AS mobile_web_pageviews,
    SUM(CASE WHEN access_method = "mobile app" THEN view_count END)
        / SUM(view_COUNT) AS mobile_app_pageviews
FROM wmf.projectview_hourly
WHERE
    agent_type = "user" 
    AND CONCAT(year, LPAD(month, 2, "0")) >= "{pv_start}" 
    AND CONCAT(year, LPAD(month, 2, "0")) < "{pv_end}"
GROUP BY
    IF(
        project IN ("mediawiki", "wikidata"),
        CONCAT("www.", project, ".org"),
        CONCAT(project, ".org")
    )
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [39]:
check(pv, index_col="domain_name")

,domain_name,monthly_pageviews,mobile_web_pageviews,mobile_app_pageviews
0,en.wikipedia.org,7.413413e+09,0.605818,0.021472
1,es.wikipedia.org,9.539750e+08,0.650961,0.006659
2,ja.wikipedia.org,1.103630e+09,0.653886,0.009408
3,de.wikipedia.org,9.245809e+08,0.528485,0.044935
4,fr.wikipedia.org,7.168126e+08,0.603533,0.016513
5,ru.wikipedia.org,8.256176e+08,0.571403,0.014359


In [41]:
merge_in(pv, on="domain_name")

## Monthly active administrators

In [40]:
maa = wmf.spark.run("""
SELECT
    wiki AS database_code,
    SUM(monthly_active_administrators) / 12 AS monthly_active_administrators
FROM (
    SELECT
        wiki_db AS wiki,
        SUBSTR(log_timestamp, 1, 6) AS month,
        COUNT(DISTINCT log_actor) AS monthly_active_administrators
    FROM wmf_raw.mediawiki_logging
    WHERE
        log_type IN ("block", "delete", "protect", "rights")
        -- Omit the "delete_redir", "move_prot", and "autopromote" actions, which can be done by regular users
        AND log_action NOT IN ("autopromote", "delete_redir", "move_prot")
        AND log_timestamp >= "{pv_start}" 
        AND log_timestamp < "{pv_end}" 
        AND snapshot = "{snapshot}"
    GROUP BY wiki_db, SUBSTR(log_timestamp, 1, 6)
) mae
GROUP BY wiki
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [42]:
check(maa)

,database_code,monthly_active_administrators
0,enwiki,423.583333
1,eswiki,50.166667
2,jawiki,34.000000
3,dewiki,129.500000
4,frwiki,98.416667
5,ruwiki,107.750000


In [43]:
merge_in(maa)

## Monthly non-bot edits

In [44]:
mnbe = wmf.spark.run("""
SELECT
   CONCAT(project, ".org") AS domain_name,
   SUM(edit_count) / 12 AS monthly_nonbot_edits
FROM wmf.edit_hourly
WHERE
    ts >= "{start}"
    AND ts < "{end}"
    AND NOT user_is_bot
    AND snapshot = "{snapshot}"
GROUP BY CONCAT(project, ".org")
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [45]:
check(mnbe, index_col="domain_name")

,domain_name,monthly_nonbot_edits
0,en.wikipedia.org,4.500923e+06
1,es.wikipedia.org,6.405978e+05
2,ja.wikipedia.org,4.546352e+05
3,de.wikipedia.org,7.821591e+05
4,fr.wikipedia.org,7.381616e+05
5,ru.wikipedia.org,5.066923e+05


In [46]:
merge_in(mnbe, on="domain_name")

## Edits Gini coefficient

In [ ]:
user_edits = wmf.spark.run("""
SELECT
    wiki_db AS wiki,
    COUNT(*) AS user_edits
FROM
    wmf.mediawiki_history
WHERE
    event_entity = "revision" 
    AND event_type = "create" 
    AND snapshot = "{snapshot}" 
    AND event_timestamp >= "{start}" 
    AND event_timestamp < "{end}" 
    AND SIZE(event_user_is_bot_by_historical) = 0
GROUP BY
    wiki_db,
    event_user_id
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
[Stage 29:==============>                                  (3572 + 160) / 12288]

In [49]:
# FROM https://github.com/oliviaguest/gini
def gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq:
    # http://www.statsdirect.com/help/generatedimages/equations/equation154.svg
    # FROM:
    # http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    # All values are treated equally, arrays must be 1d:
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array = array + 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n - 1) * array)) / (n * np.sum(array)))

egc = (
    user_edits
    .groupby("wiki")
    .apply(lambda g: gini(g["user_edits"].values))
    .reset_index()
)

egc.columns = ["database_code", "edits_Gini_coefficient"]

In [50]:
check(egc)

,database_code,edits_Gini_coefficient
0,enwiki,0.954271
1,eswiki,0.958810
2,jawiki,0.938387
3,dewiki,0.959961
4,frwiki,0.959200
5,ruwiki,0.960198


In [51]:
merge_in(egc)

## Monthly editors

In [52]:
me = wmf.spark.run("""
SELECT
    wiki AS database_code,
    COUNT(*) / 12 AS monthly_editors
FROM neilpquinn.editor_month
WHERE
    month >= "{start}" 
    AND month < "{end}" 
    AND user_id != 0 
    AND NOT bot_by_group
    AND (
        user_name NOT REGEXP "bot\\b"
        OR user_name IN ("Paucabot", "Niabot", "Marbot")
    )
GROUP BY wiki
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [53]:
check(me)

,database_code,monthly_editors
0,enwiki,135505.666667
1,eswiki,17713.916667
2,jawiki,15551.416667
3,dewiki,19901.333333
4,frwiki,20209.416667
5,ruwiki,12347.333333


In [54]:
merge_in(me)

## Unique devices per editor

In [55]:
wikis["unique_devices_per_editor"] = wikis["monthly_unique_devices"] / wikis["monthly_editors"]

In [56]:
wikis = wikis.replace([np.inf], 0)

## Content pages

In [57]:
wikis_list = wikis["database_code"].tolist()

In [58]:
ac = wmf.mariadb.run("""
SELECT
    database() AS database_code,
    ss_good_articles AS content_pages
FROM site_stats
""", wikis_list)

In [59]:
check(ac)

,database_code,content_pages
0,enwiki,6441840
1,eswiki,1746919
2,jawiki,1310090
3,dewiki,2655816
4,frwiki,2391170
5,ruwiki,1787927


In [60]:
merge_in(ac)

## Cumulative content edits

In [61]:
cce = wmf.spark.run("""
SELECT
    wiki_db AS database_code,
    COUNT(*) AS cumulative_content_edits
FROM
    wmf.mediawiki_history
WHERE
    event_entity = "revision" 
    AND event_type = "create" 
    AND snapshot = "{snapshot}" 
    AND page_namespace_is_content
    AND SIZE(event_user_is_bot_by_historical) = 0 
GROUP BY wiki_db
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [62]:
check(cce)

,database_code,cumulative_content_edits
0,enwiki,655224211
1,eswiki,94772615
2,jawiki,66956406
3,dewiki,134182420
4,frwiki,110040129
5,ruwiki,72128426


In [63]:
merge_in(cce)

## Edits per content page

In [64]:
wikis["edits_per_content_page"] = wikis["cumulative_content_edits"] / wikis["content_pages"]

## Script direction

In [65]:
rtl_wikis = get_dblist("rtl")
rtl = pd.DataFrame({"database_code": rtl_wikis, "script_direction": "right-to-left"})
merge_in(rtl)
wikis["script_direction"] = wikis["script_direction"].replace({0: "left-to-right"})

## Monthly structured discussions messages

In [66]:
msdm = wmf.mariadb.run(
"""
SELECT
    rev_user_wiki AS database_code,
    COUNT(*) / 12 AS monthly_structured_discussions_messages
FROM flowdb.flow_revision
WHERE
    rev_change_type IN ("new-post", "reply") 
    AND date_format(FROM_UNIXTIME(
        (CONV(SUBSTRING(HEX(rev_id), 1, 12), 16, 10) >> 2) / 1000),
        "%Y-%m-%d %H:%i:%S") >= "{start}" 
    AND date_format(FROM_UNIXTIME(
        (CONV(SUBSTRING(HEX(rev_id), 1, 12), 16, 10) >> 2) / 1000),
        "%Y-%m-%d %H:%i:%S") < "{end}"
GROUP BY rev_user_wiki
""".format(**query_vars), "wikishared")

In [67]:
check(msdm)

,database_code,monthly_structured_discussions_messages
0,enwiki,NaN
1,eswiki,NaN
2,jawiki,NaN
3,dewiki,NaN
4,frwiki,4110.9167
5,ruwiki,NaN


In [68]:
merge_in(msdm)

## Visual edits

In [69]:
ve = wmf.spark.run(
"""
SELECT
    CONCAT(project, ".org") AS domain_name,
    SUM(CASE WHEN ARRAY_CONTAINS(revision_tags, "visualeditor") THEN edit_count END)
        / SUM(edit_count) AS visual_edits
FROM wmf.edit_hourly
WHERE
    ts >= "{start}" 
    AND ts < "{end}"
    AND snapshot = "{snapshot}"
    AND NOT user_is_bot
GROUP BY CONCAT(project, ".org")
""".format(**query_vars))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [70]:
check(ve, index_col="domain_name")

,domain_name,visual_edits
0,en.wikipedia.org,0.083029
1,es.wikipedia.org,0.145526
2,ja.wikipedia.org,0.134256
3,de.wikipedia.org,0.098849
4,fr.wikipedia.org,0.157610
5,ru.wikipedia.org,0.150411


In [71]:
merge_in(ve, on="domain_name")

# Readying for output

In [72]:
wikis_formatted = (
    wikis[[
        "overall_size_rank",
        "monthly_unique_devices",
        "mobile_unique_devices",
        "monthly_pageviews",
        "mobile_web_pageviews",
        "mobile_app_pageviews",
        "unique_devices_per_editor",
        "monthly_editors",
        "monthly_active_editors",
        "monthly_active_administrators",
        "majority_mobile_editors",
        "monthly_new_active_editors",
        "second_month_new_editor_retention",
        "monthly_nonbot_edits",
        "bot_edits",
        "mobile_edits",
        "visual_edits",
        "anonymous_edits",
        "revert_rate",
        "edits_Gini_coefficient",
        "monthly_structured_discussions_messages",
        "content_pages",
        "cumulative_content_edits",
        "edits_per_content_page",
        "script_direction",
        "database_code",
        "project_code",
        "language_code",
        "language_name",
        "domain_name",
        "wiki_name"
    ]].astype({
        # These columns are not monthly averages and will never have a fractional component
        "overall_size_rank": int,
        "content_pages": int,
        "cumulative_content_edits": int
    })
    .sort_values("overall_size_rank")
)

# Convert underscores to spaces in column names in the final step, to avoid having to
# work with spaces beforehand
wikis_formatted = wikis_formatted.rename(columns=lambda c: c.replace("_", " "))

## Output CSV

In [73]:
wikis_formatted.to_csv(
    f"snapshots/{file_stem}.csv",
    float_format="%0.4f",
    index=False
)

## Output XLSX spreadsheet

In [ ]:
# This produces some but not all of our desired formatting. Things still needed:
# * Make the number formats actually work
# * Automatically set appropriate column widths
# * Style hyperlinks blue
# * Set alternating row colors
# * Add a filter
# 
# Continuing to try to make xlsxwriter work does not seem like a good idea. 
# Google Sheets seems to have a more comprehensive and better maintained API, and it
# allows us to go directly to our desired destination. 

xslx_data = wikis_formatted.copy()

count_columns = [
    "overall_size_rank",
    "monthly_unique_devices",
    "monthly_pageviews",
    "unique_devices_per_editor",
    "monthly_editors",
    "monthly_active_editors",
    "monthly_active_administrators",
    "monthly_new_active_editors",
    "monthly_nonbot_edits",
    "monthly_structured_discussions_messages",
    "content_pages",
    "cumulative_content_edits",
    "edits_per_content_page",
]

proportion_columns = [
    "mobile_unique_devices",
    "mobile_web_pageviews",
    "mobile_app_pageviews",
    "majority_mobile_editors",
    "second_month_new_editor_retention",
    "bot_edits",
    "mobile_edits",
    "visual_edits",
    "anonymous_edits",
    "revert_rate",
    "edits_Gini_coefficient",
]

def get_column_positions(column_names):
    return xslx_data.columns.get_indexer_for(column_names)

count_column_positions = get_column_positions(count_columns)
proportion_column_positions = get_column_positions(proportion_columns)

# Helpful documentation:
# https://xlsxwriter.readthedocs.io/working_with_pandas.html
# https://xlsxwriter.readthedocs.io/example_pandas_column_formats.html
with xlsxwriter.Workbook(f'{file_stem}.xlsx') as workbook:
    worksheet = workbook.add_worksheet(file_stem)
    
    basic_settings = {
        "font_name": "Arial",
        "font_size": 10,
        "align": "left"
    }
    basic_format = workbook.add_format(basic_settings)

    def add_derived_format(extra_settings):
        settings = basic_settings.copy()
        settings.update(extra_settings)
        return workbook.add_format(settings)
    
    header_format = add_derived_format({
        "bold": True,
        "text_wrap": True   
    })
    
    count_format = add_derived_format({
        "align": "right",
        "num_format": "#,##0"
    })
    
    percent_format = add_derived_format({
        "align": "right",
        "num_format": "0.0%"
    })

    # Turn the wiki name column into a link formula
    wiki_name = (
        '=HYPERLINK("https://'
        + wikis_formatted["domain name"]
        + '", "'
        + wikis_formatted["wiki name"]
        + '")'
    )

    xlsx_data = wikis_formatted.drop(columns=["wiki name"])
    xlsx_data.insert(0, "wiki name", wiki_name)
    
    for position, name in enumerate(xlsx_data.columns.values):
        worksheet.write(0, position, name, header_format)
        
        if position in count_column_positions:
            column_format = count_format
        elif position in proportion_column_positions:
            column_format = percent_format
        else: 
            column_format = basic_format

        worksheet.write_column(1, position, xlsx_data[name], column_format)
    
    worksheet.freeze_panes(1, 1)
    worksheet.autofilter(0, 0, len(xlsx_data.index), len(xlsx_data.columns) - 1)